20163228 Yuseon Nam

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import pandas as pd
import random

# Read Files

In [2]:
#
# Read Train File	
#
size_row    = 28    # height of the image
size_col    = 28    # width of the image

train_file  = "mnist_train.csv"

handle_file = open(train_file, "r")
train_data  = handle_file.readlines()
handle_file.close()

train_num   = len(train_data)

train_list  = np.zeros((train_num, size_row * size_col), dtype=float)
train_a     = np.zeros((train_num, size_row * size_col + 1), dtype=float)
train_label = np.zeros((10, train_num), dtype=int)
train_original_label = np.zeros(train_num, dtype=int)

count = 0
label = 2
for line in train_data :
    line_data = line.split(',')
    train_original_label[count] = line_data[0]
    
    for i in range (10) :
        if (line_data[0] == str(i)) :
            train_label[i, count] = 1
        else :
            train_label[i, count] = -1
    
    # Image Vector
    im_vector = np.asfarray(line_data[1:])
    train_list[count, :] = im_vector
    
    train_a[count, 0]    = 1
    train_a[count, 1:]   = im_vector
    count += 1

In [3]:
#
# Read Test File
#
test_file   = "mnist_test.csv"

handle_file = open(test_file, "r")
test_data   = handle_file.readlines()
handle_file.close()

test_num    = len(test_data)

test_list   = np.zeros((test_num, size_row * size_col), dtype=float)
test_a      = np.zeros((test_num, size_row * size_col + 1), dtype=float)
test_label = np.zeros((10, test_num), dtype=int)
test_original_label = np.zeros(test_num, dtype=int)

count = 0
label = -2
for line in test_data :
    line_data = line.split(',')
    test_original_label[count] = line_data[0]
    
    for i in range (10) :
        if (line_data[0] == str(i)) :
            test_label[i, count] = 1
        else :
            test_label[i, count] = -1
    
    im_vector = np.asfarray(line_data[1:])
    test_list[count, :] = im_vector
    
    test_a[count, 0]    = 1
    test_a[count, 1:]   = im_vector
    count += 1

# Make Random Vector

In [4]:
k = 5000
random_vector = np.zeros((k, size_row * size_col), dtype=float)

for i in range (k) :
    for j in range(size_row * size_col) :
        random_vector[i, j] = random.gauss(0, 1)

In [ ]:
def calculate_new_a(original_img) :
    num = len(original_img)
    new_a = np.zeros((num, k+1), dtype=float)
    
    new_a[:, 0] = 1
    new_a[:, 1:] = np.dot(original_img, random_vector.T)
    
    for i in range(num) :
        for j in range (1, k+1) :
            new_a[i, j] = max(new_a[i, j], 0)
            
    return new_a

In [ ]:
new_train_a = np.zeros((train_num, k+1), dtype=float)
new_train_a = calculate_new_a(train_list)

In [ ]:
new_test_a = np.zeros((test_num, k+1), dtype=float)
new_test_a = calculate_new_a(test_list)

# Solve Least Square Problem

In [ ]:
def cal_least_square(image, label) :
    n = len(label)
    
    trans_image = image.T
    ata = np.dot(trans_image, image)
    
    re_label = label.reshape(n, 1)
    atb = np.dot(trans_image, re_label)
    
    mati_ata = np.linalg.pinv(ata)
    mat_atb  = np.asmatrix(atb)
    aia = np.dot(mati_ata, mat_atb)
    
    return aia

In [ ]:
theta = np.zeros((10, k + 1), dtype=float)
for i in range(10) :
    setaa    = cal_least_square(new_train_a, train_label[i, :])
    theta[i] = setaa.reshape(k+1)

# Calculate y and label according to argmax

In [ ]:
train_y = np.zeros((train_num, 10), dtype=float)
test_y  = np.zeros((test_num, 10) , dtype=float)

for i in range (10) :
    train_y[:, i] = np.dot(new_train_a, theta[i])
    test_y[: , i] = np.dot(new_test_a , theta[i])

In [ ]:
train_pred = np.zeros(train_num, dtype=int)
test_pred  = np.zeros(test_num,  dtype=int)

for i in range (train_num) :
    train_pred[i] = train_y[i, :].argmax()
    
for i in range (test_num) :
    test_pred[i] = test_y[i, :].argmax()

# Calculate Accuracy

## Train Data

In [ ]:
train_result = confusion_matrix(train_original_label, train_pred) 
print(train_result)

In [ ]:
# Calculate true positive rate & error rate
train_tp  = 0
train_err = 0
for i in range (10) :
    train_tp += train_result[i][i]
    
train_tp /= train_num
train_err = 1 - train_tp

In [ ]:
print("Train Data")
print("True Positive Rate : ", train_tp)
print("Error Rate         : ", train_err)

## Test Data

In [ ]:
test_result  = confusion_matrix(test_original_label , test_pred)
print(test_result)

In [ ]:
# Calculate true positive rate & error rate
test_tp  = 0
test_err = 0
for i in range (10) :
    test_tp += test_result[i][i]
    
test_tp /= test_num
test_err = 1 - test_tp

In [ ]:
print("Test Data")
print("True Positive Rate : ", test_tp)
print("Error Rate         : ", test_err)